## LightGBM + CatBoost “확률 평균(가중치 최적화)” 앙상블

#### 셀 1) 준비물(라이브러리) 가져오기 + 기본 설정

In [22]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score

import lightgbm as lgb
from catboost import CatBoostClassifier

# "랜덤"이 매번 바뀌면 결과가 달라져서 헷갈립니다.
# 숫자를 하나 정해두면(=SEED) 매번 비슷한 결과가 나오게 만들 수 있어요.
SEED = 42

# 내 파일 경로(당신 파일명)
DATA_PATH = "data_total_v3.csv"

# 우리가 맞추고 싶은 정답 컬럼(타깃)
TARGET = "is_churn"

# 유저 ID 컬럼(모델이 외우면 안 되는 값)
ID_COL = "msno"


#### 셀 2) CSV 읽고, 컬럼명 깨끗하게 정리하기

In [23]:
# CSV 파일 읽기
df = pd.read_csv(DATA_PATH)

# 컬럼명에 숨어있는 이상한 문자(공백/BOM 등)가 있으면
# drop이 안 되는 문제가 생길 수 있어서, 컬럼명을 싹 정리해줍니다.
df.columns = (
    pd.Index(df.columns)
    .map(lambda x: str(x))                 # 무조건 문자열로 바꾸기
    .str.replace("\ufeff", "", regex=False) # BOM 제거
    .str.replace("\u00a0", " ", regex=False)# 이상한 공백(NBSP)을 일반 공백으로
    .str.strip()                            # 앞뒤 공백 제거
)

# 확인용 출력
print("전체 컬럼 개수:", len(df.columns))
print("마지막 컬럼:", df.columns[-1])

전체 컬럼 개수: 85
마지막 컬럼: is_churn


#### 셀 3) X(문제)와 y(정답) 만들기 + msno 빼기

In [24]:
# y = 정답(0/1)
y = df[TARGET].astype(int)

# X = 문제(정답 제외한 나머지 정보들)
# - msno는 ID라서 모델이 외우면 안 좋습니다(과적합/누수 위험).
# - errors="ignore"를 쓰면 msno가 없어도 에러 없이 넘어갑니다.
X = df.drop(columns=[TARGET, ID_COL], errors="ignore")

# 무한대 값이 혹시 있으면 NaN으로 바꿉니다(안전장치)
X = X.replace([np.inf, -np.inf], np.nan)

# 진짜 msno가 빠졌는지 확인
print("msno가 X에 남아있나?", "msno" in X.columns)
print("X 모양(행,열):", X.shape)
print("y 모양:", y.shape)

msno가 X에 남아있나? False
X 모양(행,열): (860966, 83)
y 모양: (860966,)


#### 셀 4) 범주형(카테고리) 컬럼 지정 + 데이터 나누기

In [25]:
# "숫자처럼 보이지만 사실은 코드(카테고리, 범주)"인 컬럼들입니다.
# CatBoost는 이걸 꼭 알려주는 게 성능에 중요합니다.
cat_cols = [c for c in ["city", "gender", "registered_via", "last_payment_method"] if c in X.columns]
print("범주형 컬럼 후보:", cat_cols)

# 데이터를 3개로 나눕니다.
# - train: 모델이 공부하는 용도
# - valid: 중간 점검(과외 선생님이 채점하는 느낌)
# - test : 마지막 시험(최종 성적)

# 1) 전체 -> train 70%, temp 30%
X_tr, X_tmp, y_tr, y_tmp = train_test_split(
    X, y,
    test_size=0.30,
    random_state=SEED,
    stratify=y
)

# 2) temp 30% -> valid 15%, test 15% (temp를 반반)
X_va, X_te, y_va, y_te = train_test_split(
    X_tmp, y_tmp,
    test_size=0.50,
    random_state=SEED,
    stratify=y_tmp
)

print("train:", X_tr.shape, "valid:", X_va.shape, "test:", X_te.shape)

# 이탈(1)이 적은 데이터라서(불균형) 1을 더 중요하게 보도록 가중치를 만듭니다.
pos = (y_tr == 1).sum()
neg = (y_tr == 0).sum()
scale_pos_weight = neg / max(pos, 1)
print("scale_pos_weight =", scale_pos_weight)

범주형 컬럼 후보: ['city', 'gender', 'registered_via', 'last_payment_method']
train: (602676, 83) valid: (129145, 83) test: (129145, 83)
scale_pos_weight = 9.570666853755219


#### 셀 5) LightGBM 학습하기

In [26]:
# LightGBM은 범주형을 category 타입으로 주면 잘 처리합니다.
X_tr_lgb = X_tr.copy()
X_va_lgb = X_va.copy()
X_te_lgb = X_te.copy()

for c in cat_cols:
    if c == "last_payment_method":
        # 결제수단은 결측이 있을 수 있어서 -1로 채우고 정수 코드로 만든 뒤 category로
        for _X in [X_tr_lgb, X_va_lgb, X_te_lgb]:
            _X[c] = pd.to_numeric(_X[c], errors="coerce").fillna(-1).round().astype(int).astype("category")
    elif c == "gender":
        # gender는 문자열이라 unknown으로 채우고 category로
        for _X in [X_tr_lgb, X_va_lgb, X_te_lgb]:
            _X[c] = _X[c].fillna("unknown").astype("category")
    else:
        # city, registered_via 같은 코드 컬럼도 category로
        for _X in [X_tr_lgb, X_va_lgb, X_te_lgb]:
            _X[c] = _X[c].astype("category")

# LightGBM 모델 만들기(설정값은 "기본 + 적당히 안전" 수준)
lgbm = lgb.LGBMClassifier(
    objective="binary",       # 문제 유형은 이진 분류(0/1)
    n_estimators=5000,        # 최대 나무 개수(그냥 많이 잡고 early stopping으로 멈춤)
    learning_rate=0.03,
    num_leaves=64,             # 한 트리 안에서 최대로 나뉠수 있는 leaf수
    min_child_samples=50,      # 너무 잘게 쪼개는걸 방지. 데이터가 최소 50개는 되어야 분할
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    scale_pos_weight=scale_pos_weight,  # 불균형 보정
    random_state=SEED,
    n_jobs=-1
)



##### LightGBM 파라미터 해석

###### `objective="binary"`
- 문제 유형을 지정합니다.
- 이진 분류(0/1) 문제라는 뜻입니다. (`is_churn` 예측)

###### `n_estimators=5000`
- 만들 트리(부스팅 라운드)의 최대 개수입니다.
- 다만 우리는 `early stopping`을 켜서, 실제로는 5000까지 다 안 가고 중간에 멈춥니다.
- 지금 케이스에서는 “최적이 78”이었으니 사실상 5000은 상한선일 뿐입니다.

###### `learning_rate=0.03`
- 한 번 업데이트할 때 얼마나 조금씩 배울지(스텝 크기)입니다.
- 값이 작으면:
  - 학습이 천천히(안전하게) 진행되고 과적합이 줄 수 있지만,
  - 대신 더 많은 트리가 필요할 수 있습니다.
- 0.03은 비교적 “작게 천천히”에 속합니다.

###### `num_leaves=64`
- 각 트리의 복잡도(표현력)를 결정하는 중요한 값입니다.
- “한 트리 안에서 최대로 나뉠 수 있는 잎(leaf) 수”라고 이해하면 됩니다.
- 값이 클수록:
  - 복잡한 패턴을 더 잘 맞추지만
  - 과적합 위험도 커집니다.

###### `min_child_samples=50`
- 너무 잘게 쪼개서 “몇 개 안 되는 데이터”에 맞추는 걸 막는 안전장치입니다.
- 하나의 leaf(끝 노드)에 들어가는 데이터가 최소 50개는 되어야 분할을 허용합니다.
- 값이 크면 과적합이 줄고, 작으면 더 세밀하게 나눕니다.

###### `subsample=0.8`
- 각 트리를 만들 때 훈련 데이터의 80%만 랜덤으로 골라 학습합니다.
- 이렇게 하면 트리들이 조금씩 다르게 학습해서 과적합이 줄고 일반화가 좋아질 수 있습니다.
- (배깅 느낌)

###### `colsample_bytree=0.8`
- 각 트리를 만들 때 피처(컬럼)의 80%만 랜덤으로 골라 학습합니다.
- 이것도 과적합 방지 + 다양한 트리를 만드는 효과가 있습니다.

###### `reg_lambda=1.0`
- L2 정규화(가중치 페널티)입니다.
- 모델이 너무 과하게 복잡해지는 것을 억제합니다.
- 값이 커질수록 더 보수적으로(덜 복잡하게) 학습합니다.

###### `scale_pos_weight=scale_pos_weight`
- 불균형 데이터에서 **양성(1=churn)**을 더 중요하게 보도록 가중치를 주는 값입니다.
- 일반적으로 `neg/pos` 정도로 설정합니다. (당신 코드도 그렇게 계산)
- 이 값을 주면 “1을 놓치지 않도록” 학습 성향이 바뀝니다.

###### `random_state=SEED`
- 랜덤 요소(데이터 샘플링 등)를 고정해서 재현 가능하게 만듭니다.

###### `n_jobs=-1`
- CPU 코어를 가능한 한 전부 사용해서 빠르게 학습합니다.

In [27]:
# LightGBM 학습 시작!
# early_stopping: valid 성능이 더 안 좋아지면 자동으로 멈춰서 과적합을 줄임
lgbm.fit(
    X_tr_lgb, y_tr,
    eval_set=[(X_va_lgb, y_va)],
    eval_metric="auc",
    callbacks=[
        lgb.early_stopping(stopping_rounds=200),
        lgb.log_evaluation(period=200),
    ],
)

# 확률 예측(0~1 사이): "이탈일 확률"
p_va_lgb = lgbm.predict_proba(X_va_lgb)[:, 1]
p_te_lgb = lgbm.predict_proba(X_te_lgb)[:, 1]


[LightGBM] [Info] Number of positive: 57014, number of negative: 545662
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.176936 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16082
[LightGBM] [Info] Number of data points in the train set: 602676, number of used features: 83
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.094601 -> initscore=-2.258703
[LightGBM] [Info] Start training from score -2.258703
Training until validation scores don't improve for 200 rounds
[200]	valid_0's auc: 0.988908	valid_0's binary_logloss: 0.131362
Early stopping, best iteration is:
[78]	valid_0's auc: 0.988399	valid_0's binary_logloss: 0.12921


##### LightGBM 학습 로그 해석

###### [LightGBM] [Info] Number of positive: 57014, number of negative: 545662  
- 학습용(train) 데이터에서  
  - positive(1 = churn) 개수 = 57,014  
  - negative(0 = non-churn) 개수 = 545,662  
- 즉, 이탈(1)이 적은 **불균형 데이터**라는 걸 보여줍니다.

###### [LightGBM] [Info] Total Bins 16082  
- LightGBM은 연속형 값을 그대로 쓰기보다 **값 구간(bin)** 으로 쪼개서(히스토그램 방식) 학습합니다.  
- “전체 피처에서 만들어진 bin의 총 개수”가 16,082라는 뜻입니다.

###### [LightGBM] [Info] Number of data points in the train set: 602676, number of used features: 83  
- train 샘플 수 = 602,676개  
- 사용한 피처 수 = 83개

###### Training until validation scores don't improve for 200 rounds  
- “validation 성능이 200번 연속으로 좋아지지 않으면 멈추겠다”는 뜻입니다.  
- 즉, 과적합을 막으려고 자동으로 멈추는 장치입니다.

###### [200] valid_0's auc: 0.988908  valid_0's binary_logloss: 0.131362  
- 200번째 트리(=200번째 부스팅 라운드)까지 학습했을 때, 검증(valid) 데이터에서의 성능이  
  - AUC = 0.988908  
  - Logloss = 0.131362  
  입니다.  
- 의미  
  - AUC: 0.5면 랜덤, 1.0이면 완벽. 0.989는 매우 높음(상당히 잘 맞춘다는 뜻).  
  - binary_logloss: 확률 예측이 얼마나 정답에 가까운지를 보는 손실. **낮을수록 좋음**.

###### Early stopping, best iteration is: [78] ...  
- early stopping 규칙 때문에 학습을 계속 지켜보다가, 검증 성능이 더 이상 좋아지지 않아서 멈췄고,  
  그중에서 **가장 좋았던 시점(best iteration)**이 78번째 트리였다는 뜻입니다.

###### [78] valid_0's auc: 0.988399  valid_0's binary_logloss: 0.12921  
- 당신은 `n_estimators=5000`이라고 크게 잡았지만,  
  실제로는 **78개 트리까지만 쓰는 모델이 검증에서 가장 좋았다**는 결론입니다.


#### 셀 6) CatBoost 학습하기

In [28]:
# CatBoost는 범주형을 "cat_features"로 알려줘야 진짜 장점이 나옵니다.
X_tr_cb = X_tr.copy()
X_va_cb = X_va.copy()
X_te_cb = X_te.copy()

# CatBoost는 코드형 범주를 문자열로 바꾸면 "숫자 크기"로 오해하지 않아서 안전합니다.
for c in cat_cols:
    if c == "last_payment_method":
        for _X in [X_tr_cb, X_va_cb, X_te_cb]:
            _X[c] = pd.to_numeric(_X[c], errors="coerce").fillna(-1).round().astype(int).astype(str)
    elif c == "gender":
        for _X in [X_tr_cb, X_va_cb, X_te_cb]:
            _X[c] = _X[c].fillna("unknown").astype(str)
    else:
        for _X in [X_tr_cb, X_va_cb, X_te_cb]:
            _X[c] = _X[c].astype(str)

# CatBoost는 범주형 컬럼을 "인덱스 번호"로 주는 게 가장 안정적입니다.
cat_idx = [X_tr_cb.columns.get_loc(c) for c in cat_cols]
print("CatBoost cat_idx:", cat_idx)

cb = CatBoostClassifier(
    loss_function="Logloss",     # 이진 분류에서 흔히 쓰는 손실함수
    eval_metric="AUC",
    iterations=5000,
    learning_rate=0.05,
    depth=8,
    random_seed=SEED,
    verbose=200,
    od_type="Iter",              # overfitting detector
    od_wait=200,
    scale_pos_weight=scale_pos_weight,
)

cb.fit(
    X_tr_cb, y_tr,
    cat_features=cat_idx,
    eval_set=(X_va_cb, y_va),
    use_best_model=True
)

p_va_cb = cb.predict_proba(X_va_cb)[:, 1]
p_te_cb = cb.predict_proba(X_te_cb)[:, 1]


CatBoost cat_idx: [0, 1, 2, 74]
0:	test: 0.9690029	best: 0.9690029 (0)	total: 509ms	remaining: 42m 23s
200:	test: 0.9882939	best: 0.9882963 (194)	total: 1m 33s	remaining: 37m 5s
400:	test: 0.9886956	best: 0.9887100 (360)	total: 3m 12s	remaining: 36m 44s
600:	test: 0.9887366	best: 0.9887726 (532)	total: 4m 46s	remaining: 34m 55s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9887725949
bestIteration = 532

Shrink model to first 533 iterations.


##### CatBoost 학습 로그 해석

##### `CatBoost cat_idx: [0, 1, 2, 74]`
- CatBoost에게 “어떤 컬럼이 범주형(categorical)인지” 알려주는 정보입니다.
- `[0, 1, 2, 74]`는 `X_tr_cb` 데이터프레임에서 범주형 컬럼들이 위치한 **열 인덱스 번호**입니다.
  - 예: 0번째 열 = city, 1번째 열 = gender, ... 처럼 매핑됩니다.
- CatBoost는 이 인덱스들을 보고 해당 컬럼들을 “숫자 크기”가 아니라 “카테고리”로 처리합니다.

---

##### 진행 로그(중간중간 찍히는 줄)
예시:
- `0: test: 0.9690029 best: 0.9690029 (0) ...`
- `200: test: 0.9882939 best: 0.9882963 (194) ...`
- `400: test: 0.9886956 best: 0.9887100 (360) ...`
- `600: test: 0.9887366 best: 0.9887726 (532) ...`

각 줄의 의미:
- `0:` / `200:` / `400:` / `600:`  
  - 현재까지 학습한 **트리(부스팅 라운드) 번호**입니다.
- `test:`  
  - 여기서 test는 “진짜 테스트셋”이 아니라, `fit()`에 넣은 `eval_set`(검증셋)의 성능입니다.
  - 현재 출력은 AUC를 기준으로 표시됩니다. (eval_metric="AUC" 설정)
- `best:`  
  - 지금까지 관측된 **최고 검증 AUC** 값입니다.
- `(194)` 같은 숫자  
  - 그 `best` 성능이 나왔던 **iteration 번호**입니다.
- `total: 1m 31s`  
  - 현재까지 걸린 총 시간입니다.
- `remaining: 36m 16s`  
  - 남은 시간을 대략 추정한 값입니다(추정치라 정확하지 않을 수 있음).

---

###### `Stopped by overfitting detector (200 iterations wait)`
- 과적합 감지기(early stopping)가 작동해서 멈췄다는 뜻입니다.
- 의미: “검증 성능이 **200번 연속으로** 더 좋아지지 않아서 학습을 중단했다.”

---

##### 최종 요약
`bestTest = 0.9887725949`
- 검증셋(eval_set)에서 나온 **최고 AUC**가 0.9887725949였다는 뜻입니다.

`bestIteration = 532`
- 최고 성능이 **532번째 iteration**에서 나왔다는 뜻입니다.

`Shrink model to first 533 iterations.`
- 최종 모델은 **0~532 (총 533개)** 트리까지만 남겨서 사용한다는 뜻입니다.
- 즉, iterations를 5000으로 크게 잡았지만, 성능이 제일 좋았던 지점까지만 “잘라서” 최종 모델로 확정한 것입니다.


#### 셀 7) 앙상블(두 모델 확률을 섞어서 더 좋게 만들기)

In [29]:
# 앙상블은 "두 모델의 예측 확률을 섞어서 평균"내는 것입니다.
# 예: 0.7*LightGBM + 0.3*CatBoost 같은 식

weights = np.linspace(0, 1, 101)  # 0.00, 0.01, ... 1.00

def best_weight_by_pr_auc(y_true, p1, p2):
    # PR-AUC(AP)가 가장 좋아지는 가중치 w를 찾습니다.
    best_w, best_ap = None, -1
    for w in weights:
        p = w * p1 + (1 - w) * p2
        ap = average_precision_score(y_true, p)
        if ap > best_ap:
            best_ap = ap
            best_w = w
    return best_w, best_ap

best_w, best_ap = best_weight_by_pr_auc(y_va, p_va_lgb, p_va_cb)
print("최적 가중치 w =", best_w, "| valid PR-AUC =", best_ap)

p_va_ens = best_w * p_va_lgb + (1 - best_w) * p_va_cb
p_te_ens = best_w * p_te_lgb + (1 - best_w) * p_te_cb

"""
가중치 0.13 의미 
LightGBM 예측을 13%
CatBoost 예측을 87%
섞는 게 valid에서 PR-AUC가 가장 좋았다는 뜻입니다.
즉, “CatBoost가 메인이지만, LightGBM을 조금 섞으면 미세하게 더 좋아진다”는 결과입니다.
"""


최적 가중치 w = 0.13 | valid PR-AUC = 0.9335030473387651


'\n가중치 0.13 의미 \nLightGBM 예측을 13%\nCatBoost 예측을 87%\n섞는 게 valid에서 PR-AUC가 가장 좋았다는 뜻입니다.\n즉, “CatBoost가 메인이지만, LightGBM을 조금 섞으면 미세하게 더 좋아진다”는 결과입니다.\n'

In [30]:
def metric_line(name, y, p):
    print(f"{name:8s} | ROC-AUC={roc_auc_score(y,p):.6f} | PR-AUC={average_precision_score(y,p):.6f}")

print("---- VALID ----")
metric_line("LGBM", y_va, p_va_lgb)
metric_line("CAT",  y_va, p_va_cb)
metric_line("ENS",  y_va, p_va_ens)

print("---- TEST ----")
metric_line("LGBM", y_te, p_te_lgb)
metric_line("CAT",  y_te, p_te_cb)
metric_line("ENS",  y_te, p_te_ens)

print("---- TRAIN ----")
# LightGBM train 예측
p_tr_lgb = lgbm.predict_proba(X_tr_lgb)[:, 1]
metric_line("LGBM", y_tr, p_tr_lgb)

# CatBoost train 예측
p_tr_cb = cb.predict_proba(X_tr_cb)[:, 1]
metric_line("CAT", y_tr, p_tr_cb)

# 앙상블 train 예측(같은 w 사용)
p_tr_ens = best_w * p_tr_lgb + (1 - best_w) * p_tr_cb
metric_line("ENS", y_tr, p_tr_ens)

---- VALID ----
LGBM     | ROC-AUC=0.988399 | PR-AUC=0.925410
CAT      | ROC-AUC=0.988773 | PR-AUC=0.933299
ENS      | ROC-AUC=0.988817 | PR-AUC=0.933503
---- TEST ----
LGBM     | ROC-AUC=0.987424 | PR-AUC=0.921090
CAT      | ROC-AUC=0.987883 | PR-AUC=0.929604
ENS      | ROC-AUC=0.987917 | PR-AUC=0.929703
---- TRAIN ----
LGBM     | ROC-AUC=0.988881 | PR-AUC=0.925152
CAT      | ROC-AUC=0.991027 | PR-AUC=0.940619
ENS      | ROC-AUC=0.990925 | PR-AUC=0.939980


#### 셀 8) 성적표 출력 + 예측 저장

In [31]:
def report(name, y_true, p):
    auc = roc_auc_score(y_true, p)
    ap  = average_precision_score(y_true, p)
    print(f"[{name}] ROC-AUC={auc:.5f} | PR-AUC(AP)={ap:.5f}")

print("\n--- VALID(중간 점검) ---")
report("LGBM", y_va, p_va_lgb)
report("CB  ", y_va, p_va_cb)
report(f"ENS(w={best_w:.2f})", y_va, p_va_ens)

print("\n--- TEST(최종 시험) ---")
report("LGBM", y_te, p_te_lgb)
report("CB  ", y_te, p_te_cb)
report(f"ENS(w={best_w:.2f})", y_te, p_te_ens)

# -------------------------
# 예측 결과를 print로 보기
# -------------------------
out = pd.DataFrame({
    "p_lgbm": p_te_lgb,
    "p_catboost": p_te_cb,
    "p_ensemble": p_te_ens,
    "y_true": y_te.values
})

print("\n--- TEST 예측 결과 샘플(상위 20개) ---")
print(out.head(20))

print("\n--- p_ensemble 상위 20개(이탈 확률 높은 순) ---")
print(out.sort_values("p_ensemble", ascending=False).head(20))

print("\n--- p_ensemble 하위 20개(이탈 확률 낮은 순) ---")
print(out.sort_values("p_ensemble", ascending=True).head(20))


--- VALID(중간 점검) ---
[LGBM] ROC-AUC=0.98840 | PR-AUC(AP)=0.92541
[CB  ] ROC-AUC=0.98877 | PR-AUC(AP)=0.93330
[ENS(w=0.13)] ROC-AUC=0.98882 | PR-AUC(AP)=0.93350

--- TEST(최종 시험) ---
[LGBM] ROC-AUC=0.98742 | PR-AUC(AP)=0.92109
[CB  ] ROC-AUC=0.98788 | PR-AUC(AP)=0.92960
[ENS(w=0.13)] ROC-AUC=0.98792 | PR-AUC(AP)=0.92970

--- TEST 예측 결과 샘플(상위 20개) ---
      p_lgbm  p_catboost  p_ensemble  y_true
0   0.098069    0.071990    0.075380       0
1   0.877110    0.991890    0.976969       1
2   0.011306    0.002322    0.003490       0
3   0.026776    0.008388    0.010779       0
4   0.041551    0.035004    0.035855       0
5   0.011306    0.002735    0.003849       0
6   0.734473    0.818999    0.808011       0
7   0.010288    0.001808    0.002910       0
8   0.010709    0.001047    0.002303       0
9   0.075073    0.062402    0.064049       0
10  0.011259    0.001417    0.002696       0
11  0.011287    0.005899    0.006599       0
12  0.913867    0.999953    0.988762       1
13  0.028112    0.